In [37]:
from idlelib.runscript import indent_message

from dotenv import load_dotenv
from sqlalchemy.sql.elements import AnnotatedColumnElement
from tensorflow.python.ops.summary_ops_v2 import graph

load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import interrupt, Command

In [38]:
from typing_extensions import Annotated
from langchain.chat_models import init_chat_model

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

llm = init_chat_model("google_genai:gemini-2.0-flash")


class State(TypedDict):
    messages: Annotated[list, add_messages]

In [39]:
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the stock symbol
    :param symbol: stock symbol
    :return: current price of the stock
    '''
    return {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "AMZN": 150.0,
        "RIL": 87.6
    }.get(symbol, 0.0)

@tool
def buy_stocks(symbol: str, quantity: int, total_price: float) -> str:
    '''Buy stocks given the stock symbol and quantity'''
    decision = interrupt(f"Approve buying {quantity} {symbol} stocks for ${total_price:.2f}?")

    if decision == "yes":
     return f"You bought {quantity} shares of {symbol} for a total price of {total_price}"
    else:
        return "Buying declined."

tools = [get_stock_price, buy_stocks]

llm_with_tools = llm.bind_tools(tools)

In [40]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)

builder.add_node(chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START,"chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

graph = builder.compile(checkpointer=memory)


In [41]:
config = {"configurable": {"thread_id": "buy_thread"}}

'''Step1 user asks price'''
state = graph.invoke({"messages": [{"role": "user","content": "What is the current price of 10 MSFT stocks?"}]}, config=config)
print(state["messages"][-1].content)

The current price of one MSFT stock is $200.3. Therefore, the current price of 10 MSFT stocks is $2003.0.


In [44]:
'''Step2 user asks to buy '''
state = graph.invoke({"messages": [{"role": "user","content": "Buy 10 MSFT stocks at current price."}]}, config=config)
print(state.get("__interrupt__"))

decision = input("Approve (yes/no): ")
state = graph.invoke(Command(resume=decision), config=config)
print(state["messages"][-1].content)

[Interrupt(value='Approve buying 10 MSFT stocks for $2003.00?', id='af876627a63b26e5bde8a4eba96784fa')]
OK. You bought 10 shares of MSFT for a total price of $2003.0.
